In [1]:
from __future__ import division
%load_ext autoreload
%autoreload 2
from django.conf import settings
import cPickle as pickle
import csv
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('c:\dev\opinion\opinion\python\opinion')
import utils.fileutils as fileutils
import utils.metrics as metrics
import nltk

In [5]:
def computePerformanceOverallOfLearner(learnerCol, infile, filenames):
        #infile = os.path.join(dirConsolidated, "ResultsConsolidated_"+algo+".xls")
        df = fileutils.readExcel(infile, "Sheet1", encoding="ISO-8859-1")
        exps = []
        gots = []
        labels = set()
        for index, row in df.iterrows():
            fname = row["File"]
            fname = fname.split("_")[0]
            if fname not in filenames:
                #print fname, " not in filenmaes"
                #return
                continue
            else:
                exp = row["ManualLabel"]
                got = row[learnerCol]
                labels.add(exp)
                exps.append(exp)
                gots.append(got)
        computer = metrics.PerformanceMultiClass(exps, gots, labels = list(labels))
        for label in labels:
            pr = computer.precision(label)
            re = computer.recall(label)
            f1 = 2*pr*re/(pr+re)
            print "Label = %s. Precision = %.3f. Recall = %.3f. F1 = %.3f"%(label, pr, re, f1)
        f1_macro = computer.f1_macro_average()
        pr_macro = computer.precision_macro_average()
        rec_macro = computer.recall_macro_average()
        f1_micro, _, _ = computer.compute_micro_average()
        print "F1 Macro = %.3f. Micro = %.3f"%(f1_macro, f1_micro)
        print "Macro Precision = %.3f. Recall = %.3f"%(pr_macro, rec_macro)
        print "-------------------------------"


def computePerformancOfLearner(learnerCol, infile, filenames):
        #infile = os.path.join(dirConsolidated, "ResultsConsolidated_"+algo+".xls")
        df = fileutils.readExcel(infile, "Sheet1", encoding="ISO-8859-1")
        
        
        exps = dict()
        gots = dict()
        labels = dict()
        for index, row in df.iterrows():
            fname = row["File"]
            fname = fname.split("_")[0]
            if fname not in filenames:
                #print fname, " not in filenmaes"
                continue
            else:
                if fname not in exps:
                    exps[fname] = []
                    gots[fname] = []
                    labels[fname] = set()
            exp = row["ManualLabel"]
            got = row[learnerCol]
            labels[fname].add(exp)
            exps[fname].append(exp)
            gots[fname].append(got)
        for fname in filenames:
            computer = metrics.PerformanceMultiClass(exps[fname], gots[fname], labels = list(labels[fname]))
            for label in labels[fname]:
                pr = computer.precision(label)
                re = computer.recall(label)
                f1 = 2*pr*re/(pr+re)
                print "File %s. Label = %s. Precision = %.2f. Recall = %.2f. F1 = %.2f"%(fname, label, pr, re, f1)
            f1_macro = computer.f1_macro_average()
            f1_micro, _, _ = computer.compute_micro_average()
            print "File = %s. F1 Macro = %.2f. Micro = %.2f"%(fname, f1_macro, f1_micro)
            print "-------------------------------"


In [3]:
indir = r"C:\dev\opinion\papers\sentiplus\Hybrid"
infile = os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.xls")

In [6]:
leanerCol = "POME"
filenames = ["DatasetLinJIRA", "BenchmarkUddinSO", "DatasetLinAppReviews", 
                     "DatasetLinSO", "DatasetSenti4SDSO", "OrtuJIRA"]

In [7]:
computePerformanceOverallOfLearner(leanerCol, infile, filenames)

Label = p. Precision = 0.389. Recall = 0.076. F1 = 0.127
Label = o. Precision = 0.550. Recall = 0.950. F1 = 0.697
Label = n. Precision = 0.415. Recall = 0.035. F1 = 0.065
F1 Macro = 0.397. Micro = 0.540
Macro Precision = 0.451. Recall = 0.354
-------------------------------


In [8]:
computePerformancOfLearner(leanerCol, infile, filenames)

File DatasetLinJIRA. Label = p. Precision = 0.43. Recall = 0.86. F1 = 0.57
File DatasetLinJIRA. Label = n. Precision = 0.86. Recall = 0.43. F1 = 0.57
File = DatasetLinJIRA. F1 Macro = 0.64. Micro = 0.57
-------------------------------
File BenchmarkUddinSO. Label = p. Precision = 0.41. Recall = 0.12. F1 = 0.19
File BenchmarkUddinSO. Label = o. Precision = 0.60. Recall = 0.94. F1 = 0.73
File BenchmarkUddinSO. Label = n. Precision = 0.45. Recall = 0.05. F1 = 0.10
File = BenchmarkUddinSO. F1 Macro = 0.42. Micro = 0.58
-------------------------------
File DatasetLinAppReviews. Label = p. Precision = 0.40. Recall = 0.03. F1 = 0.06
File DatasetLinAppReviews. Label = o. Precision = 0.07. Recall = 0.84. F1 = 0.12
File DatasetLinAppReviews. Label = n. Precision = 0.67. Recall = 0.05. F1 = 0.09
File = DatasetLinAppReviews. F1 Macro = 0.34. Micro = 0.10
-------------------------------
File DatasetLinSO. Label = p. Precision = 0.42. Recall = 0.15. F1 = 0.22
File DatasetLinSO. Label = o. Precision 

In [10]:
with open(os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.pkl")) as f:
    df = pickle.load(f)
#df = fileutils.readExcel(infile, "Sheet1", encoding="ISO-8859-1")
#for index, row in df.iterrows():
df.columns    

Index([             u'Adaptive_CLF_ADB',              u'Adaptive_CLF_GBT',
                     u'Adaptive_CLF_RF',            u'Adaptive_POLAR_ADB',
       u'Adaptive_POLAR_ADB_HotEncoded',            u'Adaptive_POLAR_GBT',
       u'Adaptive_POLAR_GBT_HotEncoded',             u'Adaptive_POLAR_RF',
        u'Adaptive_POLAR_RF_HotEncoded',                          u'Best',
                              u'BestID',              u'DsoLabelFullText',
         u'DsoLabelFullText_HotEncoded',                          u'File',
                         u'ManualLabel',        u'ManualLabel_HotEncoded',
                     u'Nscore_FullText',               u'Pscore_FullTest',
                             u'SEScore',                           u'SID',
                            u'Sentence',                      u'Senti4SD',
                 u'Senti4SD_HotEncoded',                       u'SentiCR',
                  u'SentiCR_HotEncoded',               u'SentistrengthSE',
          u'Sentistrength

In [13]:
ones = 0
twos = 0
for index, row in df.iterrows():
    pome_res = row["POME_RES"]
    senti = pome_res["sentiment"]
    if senti != "":
        print senti
    

POS 
POS 
POS 
POS 
NEG 
NEG 
POS 
POS 
POS 
POS 
POS 
NEG 
POS 
POS 
NEG
POS 
POS 
POS 
POS 
NEG 
POS 
NEG 
POS 
POS 
NEG 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
NEG 
POS 
NEG 
NEG 
NEG 
NEG 
POS 
NEG 
POS 
POS 
POS 
NEG 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
NEG 
NEG 
POS 
POS 
POS 
NEG 
POS 
POS 
POS 
NEG 
POS 
POS 
POS 
NEG 
NEG 
POS 
POS 
POS 
POS 
NEG 
POS 
POS 
POS POS 
POS 
POS 
POS 
POS
POS 
NEG 
POS 
POS 
POS 
POS 
POS 
POS 
NEG 
NEG 
NEG 
NEG 
NEG 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
NEG 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS POS 
POS 
POS 
POS 
POS 
POS 
POS 
NEG 
NEG 
POS 
POS 
NEG 
POS 
NEG 
NEG POS 
POS 
POS 
NEG 
NEG 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
NEG 
POS 
POS 
POS 
NEG 
POS 
POS 
NEG 
POS 
POS 
POS 
POS 
POS 
NEG 
POS 
POS 
POS 
POS 
POS 
NEG 
NEG 
NEG 
POS 
POS 
POS 
POS 
POS 
NEG 
POS 
NEG 
POS 
POS POS 
POS 
POS 
POS 
POS 
POS 
POS 
NEG NEG 
POS 
POS 
POS 
POS 
POS 
POS 
POS 
NEG NEG 
NEG 
NEG 
POS 
POS 
POS 
POS